In [1]:
import camelot
import pandas as pd

In [2]:
# Read the PDF file
pdf_path = "main_.pdf"
tables = camelot.read_pdf(pdf_path, pages="all", flavor='stream')  # Use 'lattice' if the tables have borders, 'stream' if not

# Check how many tables were found
print(f"Total tables found: {len(tables)}")

/Users/aaravanand/anaconda3/lib/python3.11/site-packages/camelot/parsers/stream.py:365: UserWarning: No tables found in table area 1
  warnings.warn(f"No tables found in table area {table_idx + 1}")


Total tables found: 120


In [4]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
excel_path = "extracted_tables.xlsx"
writer = pd.ExcelWriter(excel_path, engine='xlsxwriter')
# Get the workbook and the worksheet objects
workbook = writer.book
# Define the format for the headers
header_format = workbook.add_format({
    'bold': True,
    'bg_color': '#D7E4BC',  # Light green background
    'font_color': '#000000',  # Black text
    'border': 1
})
# Define the format for the cell width
cell_format = workbook.add_format({
    'text_wrap': True,
    'border': 1
})
def is_header_row(row):
    # Simple heuristic: check if the majority of the cells in the row are strings
    return sum(isinstance(cell, str) for cell in row) > (len(row) / 2)
# Loop through the tables and save each one as a separate sheet
for i, table in enumerate(tables):
    df = table.df
    # Remove the first row
    df = df.iloc[1:].reset_index(drop=True)
    # Detect the header row
    header_row = None
    for j in range(min(5, len(df))):  # Check first 5 rows to find a suitable header
        if is_header_row(df.iloc[j]):
            header_row = df.iloc[j]
            df = df.iloc[j+1:].reset_index(drop=True)
            df.columns = header_row
            break
    if header_row is None:
        df.columns = [f'Column_{k}' for k in range(len(df.columns))]
    # Write the dataframe to Excel without the original header row
    df.to_excel(writer, sheet_name=f'Table_{i+1}', index=False, startrow=1, header=False)
    worksheet = writer.sheets[f'Table_{i+1}']
    # Get the dimensions of the dataframe
    (max_row, max_col) = df.shape
    # Write the column headers with the defined format
    for col_num, value in enumerate(df.columns.values):
        worksheet.write(0, col_num, value, header_format)
    # Apply the cell format to all cells
    for row in range(1, max_row + 1):
        for col in range(max_col):
            worksheet.write(row, col, str(df.iloc[row - 1, col]), cell_format)
    # Set the column widths (MODIFIED PART)
    for col_num, value in enumerate(df.columns.values):
        try:
            max_len = df[value].astype(str).map(len).max()
            header_len = len(str(value))
            column_len = max(max_len, header_len) + 2
            worksheet.set_column(col_num, col_num, column_len)
        except Exception as e:
            print(f"Error processing column {value}: {str(e)}")
            # Set a default column width if there's an error
            worksheet.set_column(col_num, col_num, 15)
# Save the Excel file
writer.close()
print(f"Tables extracted and saved to {excel_path}")

Error processing column : The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().
Error processing column : The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().
Error processing column Group: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().
Error processing column Group: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().
Error processing column Bank: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().
Error processing column Bank: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().
Error processing column : The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().
Error processing column : The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().
Error processi